In [1]:
# Just take advantage of jupyter notebook. Could've written things in OPP.

from pathlib import Path
from datetime import date, timedelta

journal_start = date(2020, 3, 1)
one_day = timedelta(days=1)
 # not using strftime("%B") for months since it depends on the current locale
month_en = ('January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December')
weekday_zh_hant = ('週一', '週二', '週三', '週四', '週五', '週六', '週日')

In [2]:
def path_manual(y, m, d):
    return Path('manual', str(y), str(m), f'{d}.md')

def manual_content(y, m, d):
    md_file = path_manual(y, m, d)
    if Path(md_file).is_file():
        with open(md_file, 'r') as f:
            content = f.read()
        return content
    else:
        return ""
# manual_content(2021,3,1)

In [3]:
def start_str(y, m, d):
    '''Return the starting string for the page of y/m/d'''
    dt = date(y, m, d)

    front_matter = f'''---
title: {m} 月 {d} 號（{weekday_zh_hant[dt.weekday()]}）
description: {y} 年 {m} 月 {d} 日法國新冠肺炎疫情匯報。法國 COVID-19 日誌第 {(dt - journal_start).days + 1} 篇。
---
'''
    export_import = f'''
export const date = "{dt.strftime('%Y-%m-%d')}";
import {{ Subtitle, Grace, OfficialData, SourceFb, Figure }} from "@site/src/scripts/components";
'''
    subtitle = '''
<Subtitle date={date} />
'''
    official_data='''
## 法國官方數據 {#official-data}

<OfficialData date={date} />
'''
    main_img = '''
<Figure date={date}></Figure>
'''
    news_header = '''
## 本日新聞重點 {#new}

<SourceFb date={date} />
'''
    news_header = news_header + "\n" if path_manual(y, m, d).is_file() else ""

    if dt == date(2020,3,1):
        return  front_matter + export_import + subtitle + main_img
    else:
        return front_matter + export_import + subtitle + official_data + main_img + news_header
# print(start_str(2020,3,2))

## Generate pages to the journal

Generate pages ONLY for dates after 2020/3/1 (included)

In [4]:
def generate_a_page(y, m, d):
    folder = Path('..', '..', 'docs', str(y), month_en[m-1].lower())
    file_name = f'{d}.mdx'
    mdx = Path(folder, file_name)

    man = manual_content(y, m, d)
    mdx_content = start_str(y, m, d) + man # start_str will make sure the input `y, m, d` is of right format 
    prompt_note = "" if man else " (without manual content)"

    if Path(mdx).is_file():
        print(f'- overwriting mdx of {y}/{m}/{d} in `docs`{prompt_note}')
    else:
        print(f'- creating mdx for {y}/{m}/{d} in `docs`{prompt_note}')

    # write file
    folder.mkdir(parents=True, exist_ok=True)
    with open(mdx, 'w') as f:
        f.write(mdx_content)

def generate_pages(s_y, s_m, s_d, e_y, e_m, e_d):
    s_dt = date(s_y, s_m, s_d)
    e_dt = date(e_y, e_m, e_d)
    n_pages = (e_dt - s_dt).days + 1
    if n_pages < 2:
        raise Exception("The ending date should be at least one day after the starting day.")

    prompt = f'writing {n_pages} pages: from {s_y}/{s_m}/{s_d} to {e_y}/{e_m}/{e_d}'
    print(f'Start {prompt}')

    dt = s_dt
    while (dt != e_dt + one_day):
        generate_a_page(dt.year, dt.month, dt.day)
        dt += one_day

    print(f'Finish {prompt}')

In [5]:
generate_a_page(2020,3,1)
generate_a_page(2020,12,31)
generate_a_page(2021,2,1)
generate_pages(2021,3,1,2021,3,10)

- creating mdx for 2020/3/1 in `docs` (without manual content)
- creating mdx for 2020/12/31 in `docs` (without manual content)
- creating mdx for 2021/2/1 in `docs` (without manual content)
Start writing 10 pages: from 2021/3/1 to 2021/3/10
- creating mdx for 2021/3/1 in `docs`
- creating mdx for 2021/3/2 in `docs` (without manual content)
- creating mdx for 2021/3/3 in `docs` (without manual content)
- creating mdx for 2021/3/4 in `docs` (without manual content)
- creating mdx for 2021/3/5 in `docs` (without manual content)
- creating mdx for 2021/3/6 in `docs` (without manual content)
- creating mdx for 2021/3/7 in `docs` (without manual content)
- creating mdx for 2021/3/8 in `docs` (without manual content)
- creating mdx for 2021/3/9 in `docs` (without manual content)
- creating mdx for 2021/3/10 in `docs` (without manual content)
Finish writing 10 pages: from 2021/3/1 to 2021/3/10
